In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
top_authors = df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

min_articles = min(top_authors["title"])
adf = pd.concat([df[df["author"] == author].sample(min_articles, random_state=42)
                     for author in top_authors.index.values])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
tfidf_vectors = tfidf_vectorizer.fit_transform(adf["nav"])
tfidf_vectors

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(random_state=42)
scores = cross_val_score(clf, tfidf_vectors, adf["author"].values, scoring="accuracy")
scores

In [ ]:
cross_val_score(clf, tfidf_vectors, adf["author"].values, scoring="precision_macro")

In [ ]:
from sklearn.model_selection import cross_validate
clf = SGDClassifier(random_state=42)
cv = cross_validate(clf, tfidf_vectors, y=adf["author"].values, 
                    scoring=("accuracy", "precision_weighted", "recall_weighted"))
cv

In [ ]:
print(cv['test_accuracy'].std() / cv['test_accuracy'].mean())

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

for clf_class in [SVC, SGDClassifier, MultinomialNB, 
                  DecisionTreeClassifier, RandomForestClassifier]:
    clf = clf_class(random_state=42) if clf_class != MultinomialNB else clf_class()
    print(clf_class.__name__)
    cv = cross_validate(clf, tfidf_vectors, y=adf["author"].values, 
                         scoring=("accuracy", "precision_weighted", "recall_weighted"))
    for m in ["accuracy", "precision_weighted", "recall_weighted"]:
        print(m, cv[f'test_{m}'].mean(), cv[f'test_{m}'].std() / cv[f'test_{m}'].mean())
    print("\n")